In [1]:
# 不含模型参数的自定义层
from mxnet import gluon, nd
from mxnet.gluon import nn

In [2]:
class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    
    def forward(self, x):
        return x - x.mean()

In [3]:
layer = CenteredLayer()

In [5]:
layer(nd.array([1, 2, 3, 4, 5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [6]:
# 更复杂的模型
net = nn.Sequential()
net.add(nn.Dense(128), CenteredLayer())

In [7]:
net.initialize()
y = net(nd.random.uniform(shape=(4, 8)))
y.mean().asscalar()

-7.212293e-10

In [8]:
# 含模型参数的自定义层
params = gluon.ParameterDict()
params.get('param2', shape=(2, 3))
params

(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

In [9]:
class MyDense(nn.Block):
    # units: 该层的输出个数； in_units: 该层的输入个数
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))
    
    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

In [10]:
dense = MyDense(units=3, in_units=5)
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

In [11]:
dense.initialize()

In [12]:
dense(nd.random.uniform(shape=(2, 5)))


[[ 0.06917784  0.01627153  0.01029644]
 [ 0.02602214  0.0453731   0.        ]]
<NDArray 2x3 @cpu(0)>

In [13]:
# 使用自定义层构造模型
net = nn.Sequential()
net.add(MyDense(8, in_units=64),
       MyDense(1, in_units=8))
net.initialize()
net(nd.random.uniform(shape=(2, 64)))


[[ 0.03820474]
 [ 0.04035058]]
<NDArray 2x1 @cpu(0)>